In [14]:
from time import time
from __future__ import print_function

from time import time
import logging
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [16]:
from PIL import Image
from sklearn.ensemble import RandomForestClassifier

In [44]:
from sklearn.datasets import fetch_lfw_pairs

lfw_dataset_train = fetch_lfw_pairs(subset='train', resize=0.4)
lfw_dataset_test = fetch_lfw_pairs(subset='test',resize=0.4)

In [45]:
lfw_dataset_test.keys()

dict_keys(['data', 'pairs', 'target', 'target_names', 'DESCR'])

In [47]:
X_train = lfw_dataset_train.data
y_train = lfw_dataset_train.target

X_test = lfw_dataset_test.data
y_test = lfw_dataset_test.target

In [48]:
cnn_train_data = []
cnn_train_data_1 = []
cnn_train_data_2 = []
for images_paris in lfw_dataset_train['pairs']:
    image1 = images_paris[0]
    image2 = images_paris[1]
    # Stack the two images along the channel axis to create a single image 
    # with dimensions (32, 32, 6)
    combined_image = np.concatenate((image1, image2), axis=0)
    # combined_image = combined_image[:, :, :3]
    cnn_train_data_1.append(image1)
    cnn_train_data_2.append(image2)
    cnn_train_data.append(combined_image)
cnn_train_data  = np.stack(cnn_train_data, axis=0)
cnn_train_data_1  = np.stack(cnn_train_data_1, axis=0)
cnn_train_data_2  = np.stack(cnn_train_data_2, axis=0)

cnn_test_data = []
cnn_test_data_1 = []
cnn_test_data_2 = []
for images_paris in lfw_dataset_test['pairs']:
    image1 = images_paris[0]
    image2 = images_paris[1]
    # Stack the two images along the channel axis to create a single image 
    # with dimensions (32, 32, 6)
    combined_image = np.concatenate((image1, image2), axis=0)
    # combined_image = combined_image[:, :, :3]
    cnn_test_data_1.append(image1)
    cnn_test_data_2.append(image2)
    cnn_test_data.append(combined_image)
cnn_test_data  = np.stack(cnn_test_data, axis=0)
cnn_test_data_1  = np.stack(cnn_test_data_1, axis=0)
cnn_test_data_2  = np.stack(cnn_test_data_2, axis=0)

In [49]:
cnn_train_data_2.shape, cnn_train_data_1.shape

((2200, 50, 37), (2200, 50, 37))

In [50]:
cnn_test_data_1.shape, cnn_test_data_2.shape

((1000, 50, 37), (1000, 50, 37))

In [51]:
### passing the single image to the function to check the result
X_train_1 = []
X_train_2 = []
for image in cnn_train_data_1:
    X_train_1.append(image.flatten())

X_train_1 = np.array(X_train_1)

for image in cnn_train_data_2:
    X_train_2.append(image.flatten())
X_train_2 = np.array(X_train_2)

In [52]:
X_train_1.shape, X_train_2.shape

((2200, 1850), (2200, 1850))

In [53]:
### passing the single image to the function to check the result
X_test_1 = []
X_test_2 = []
for image in cnn_test_data_1:
    X_test_1.append(image.flatten())

X_test_1 = np.array(X_test_1)

for image in cnn_test_data_2:
    X_test_2.append(image.flatten())
X_test_2 = np.array(X_test_2)

In [54]:
X_test_1.shape, X_test_2.shape

((1000, 1850), (1000, 1850))

In [57]:
def pca_implementation(X_train, X_test):
    # Compute a PCA (eigenfaces) on the face dataset (treated as unlabeled
    # dataset): unsupervised feature extraction / dimensionality reduction
    n_components = 150

    print("Extracting the top %d eigenfaces from %d faces"
          % (n_components, X_train.shape[0]))
    t0 = time()
    pca = PCA(n_components=n_components, svd_solver='randomized',
              whiten=True).fit(X_train)
    print("done in %0.3fs" % (time() - t0))

    eigenfaces = pca.components_.reshape((n_components, 50, 37))

    print("Projecting the input data on the eigenfaces orthonormal basis")
    t0 = time()
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    print("done in %0.3fs" % (time() - t0))
    return X_train_pca, X_test_pca
   

In [58]:
X_train_pca_1, X_test_pca_1 = pca_implementation(X_train_1, X_test_1)

Extracting the top 150 eigenfaces from 2200 faces
done in 0.774s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.037s


In [59]:
X_train_pca_2, X_test_pca_2 = pca_implementation(X_train_2, X_test_2)

Extracting the top 150 eigenfaces from 2200 faces
done in 0.625s
Projecting the input data on the eigenfaces orthonormal basis
done in 0.052s


In [62]:
X_train_combined = np.hstack((X_train_pca_1,X_train_pca_2))

In [63]:
X_train_combined.shape

(2200, 300)

In [61]:
X_test_combined = np.hstack((X_test_pca_1,X_test_pca_2))

In [64]:
X_test_combined.shape

(1000, 300)

In [77]:
import xgboost as xgb
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train_combined, y_train)

# Predict
y_pred = xgb_cl.predict(X_test_combined)

# Score
accuracy_score(y_test, y_pred)

0.531

In [78]:
from sklearn.metrics import confusion_matrix, classification_report

target = lfw_dataset_test.target
target_names = lfw_dataset_test.target_names


print(classification_report(y_pred, y_test, target_names=target_names))

                   precision    recall  f1-score   support

Different persons       0.51      0.53      0.52       479
      Same person       0.55      0.53      0.54       521

         accuracy                           0.53      1000
        macro avg       0.53      0.53      0.53      1000
     weighted avg       0.53      0.53      0.53      1000

